# Lesson 2 : LangGraph Components

In [1]:
import os
os.system("clear")
abs_path = os.path.dirname(os.path.abspath(__name__))
os.chdir(abs_path)
print(f"Working directory: <{os.getcwd()}>")

Working directory: </home/tjamil/Desktop/Agents/code_excercises/Langgraph/Ai Agents using Langgraph (dl.ai)>


In [2]:
#from dotenv import load_dotenv
#_ = load_dotenv()

In [3]:
from setup_environment_r2 import set_environment_variables
set_environment_variables("LangGraph Basics - deeplearning.ai")

reading environment variables from: </home/tjamil/Desktop/Agents/code_excercises/Langgraph/Ai Agents using Langgraph (dl.ai)>
API Keys loaded and tracing set with project name:  LangGraph Basics - deeplearning.ai


In [4]:
#%pip install -U langchain_groq langgraph

In [5]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

#from langchain_openai import ChatOpenAI
import os
from langchain_groq import ChatGroq
model = ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview", api_key=os.environ.get("GROQ_API_KEY"))

In [6]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)   # this is the name of the tool taht will be used by LLM

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [7]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [8]:
class Agent:

    def __init__(self, model, tools, system=""):
        #setting up the system message
        self.system = system
        
        #defining the graph
        graph = StateGraph(AgentState)
        
        #adding nodes (llm, action/tools)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        
        #adding conditional edges (decide if action should be taken or not, and which node to go to)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        
        #adding edges
        graph.add_edge("action", "llm")
        
        #setting entry point
        graph.set_entry_point("llm")
        
        #compiling the graph (creates a runnable graph)
        self.graph = graph.compile()
        
        #defining the tools
        self.tools = {t.name: t for t in tools}
        
        #defining the model with tools binding
        self.model = model.bind_tools(tools)

    #defining the function for conditional edge 
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    #defining 'llm node' function
    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    #defining 'action node' function
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls    
        # .too_call is attribute returned aalongside message by functiona calling llms
        
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                print(f"Result: {result}")

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        
        print("Back to the model!")
        return {'messages': results}

In [9]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

#model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [10]:
from IPython.display import Image

#Image(abot.graph.get_graph().draw_png())

In [11]:
messages = [HumanMessage(content="What is the weather in sf?. Give just suffiucient information.")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'call_tfg7', 'type': 'tool_call'}
Result: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729330322, 'localtime': '2024-10-19 02:32'}, 'current': {'last_updated_epoch': 1729330200, 'last_updated': '2024-10-19 02:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 25, 'wind_dir': 'NNE', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 27, 'cloud': 50, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 17.2, 'windchill_f': 63.0, 'heatindex_c': 17.2, 'heatindex_f': 63.0, 'dewpoint_c': 7.5, 'dewpoint_f': 45

In [12]:
result

{'messages': [HumanMessage(content='What is the weather in sf?. Give just suffiucient information.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tfg7', 'function': {'arguments': '{"query": "weather in sf"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_0904', 'function': {'arguments': '{"query": "current weather in san francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 301, 'total_tokens': 368, 'completion_time': 0.212095346, 'prompt_time': 0.022393446, 'queue_time': 0.011038464000000001, 'total_time': 0.234488792}, 'model_name': 'llama3-groq-70b-8192-tool-use-preview', 'system_fingerprint': 'fp_ee4b521143', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-649224f4-7afc-4f6d-a9b5-b25058c12011-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf

In [13]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 60.1°F (15.6°C). The wind is blowing at 3.8 mph (6.1 kph) from the NNE. The humidity is at 27%, and the cloud cover is at 50%. The wind chill is 63.0°F (17.2°C), and the heat index is also 63.0°F (17.2°C). The dew point is at 45.5°F (7.5°C), with a visibility of 9.0 miles (16.0 km). The UV index is 0.0, and the gust speed is 8.0 mph (12.9 kph).'

In [14]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_cjcn', 'type': 'tool_call'}
Result: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729330322, 'localtime': '2024-10-19 02:32'}, 'current': {'last_updated_epoch': 1729330200, 'last_updated': '2024-10-19 02:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 25, 'wind_dir': 'NNE', 'pressure_mb': 1018.0, 'pressure_in': 30.06, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 27, 'cloud': 50, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 17.2, 'windchill_f': 63.0, 'heatindex_c': 17.2, 'heatindex_f': 63.0, 'dewpoint_c': 7.

In [15]:
result['messages'][-1].content

'Based on the current weather data, in San Francisco, it is partly cloudy with a temperature of 60.1°F, and in Los Angeles, it is clear with a temperature of 61.0°F. Both cities are experiencing a calm night with minimal wind.'

In [16]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

#model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Who won the super bowl in 2024?'}, 'id': 'call_ar8y', 'type': 'tool_call'}
Result: [{'url': 'https://www.espn.com/nfl/story/_/id/39480722/49ers-chiefs-live-super-bowl-lviii-updates-moments-highlights', 'content': "With a relentless defense and opportune plays by their star quarterback -- including a pair of gutsy overtime scrambles -- the Chiefs won their third Super Bowl in five years in a 25-22 overtime victory against the San Francisco 49ers in only the second overtime game in Super Bowl history.\n Staff\nTaylor Swift supports Travis Kelce, chugs drink at Super Bowl LVIII10hTory Barron\nAfter posting a losing record in December, the Chiefs embraced an underdog, villain mentality throughout the postseason, upsetting three teams en route to their second consecutive Super Bowl title, becoming the ninth team to repeat as Super Bowl champions and first since the 2004 New England Patriots.\n ESPN\nSuper Bowl 2024 - Highlig

In [17]:
print(result['messages'][-1].content)

The Kansas City Chiefs won the Super Bowl in 2024. They are headquartered in Kansas City, Missouri. The GDP of Missouri is approximately $300 billion.
